In [1]:
import pandas as pd
import glob
import os
import warnings
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

In [2]:
# Use glob to find all Excel files starting with 'Content_'
file_paths = glob.glob('data/Content_*.xlsx')

In [3]:
sheet_name = "ENGAGEMENT"

# Clean and merge ENGAGEMENT sheets
engagement_dfs = []

for i, f in enumerate(file_paths):
    df = pd.read_excel(f, sheet_name=sheet_name)

    # keep header only from first file
    if i > 0:
        df = df.iloc[1:]

    engagement_dfs.append(df)

print(engagement_dfs[0].tail(5))

# engagement_dfs.to_excel("engagements.xlsx", index=False)



          Date  Impressions  Engagements
256  7/25/2025         1976           62
257  7/26/2025         1414           31
258  7/27/2025         1099           16
259  7/28/2025         1032            6
260  7/29/2025         3319           95


c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
merged_engagement_df = pd.concat(engagement_dfs)
date_col = merged_engagement_df.columns[0]
merged_engagement_df[date_col] = pd.to_datetime(merged_engagement_df[date_col], errors="coerce")

merged_engagement_df.to_excel("engagements.xlsx", index=False)

merged_cleaned_engagement_df = merged_engagement_df.drop_duplicates(subset=date_col)
merged_cleaned_engagement_df = merged_cleaned_engagement_df.sort_values(by=date_col)
print(merged_engagement_df.tail(5))


# merged_engagement_df.to_excel('engagement.xlsx', index=False)



          Date  Impressions  Engagements
120 2025-11-25          699           14
121 2025-11-26         2817           79
122 2025-11-27         1091           36
123 2025-11-28         1126           17
124 2025-11-29         1367           23


In [5]:
len(merged_engagement_df)

718

In [6]:
sheet_name = "FOLLOWERS"
followers_dfs = []

for f in file_paths:
    df = pd.read_excel(
        f,
        sheet_name=sheet_name,
        skiprows=2
    )

    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

    followers_dfs.append(df)

print(followers_dfs[0].head(5))

c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


        Date  New followers
0 2024-11-11              2
1 2024-11-12              2
2 2024-11-13              2
3 2024-11-14              1
4 2024-11-15              0


In [7]:
followers_merged = pd.concat(followers_dfs, ignore_index=True)
followers_merged = followers_merged.drop_duplicates(subset=date_col)
followers_merged = followers_merged.sort_values(by=date_col)
print(followers_merged.head(5))
# followers_merged.to_excel('followers.xlsx', index=False)

        Date  New followers
0 2024-11-11              2
1 2024-11-12              2
2 2024-11-13              2
3 2024-11-14              1
4 2024-11-15              0


In [8]:
len(followers_merged)

384

In [9]:
sheet_name = "DEMOGRAPHICS"

demographics_dfs = []

for i, f in enumerate(file_paths):
    df = pd.read_excel(
        f,
        sheet_name=sheet_name,
    )
    # keep header only from first file
    if i > 0:
        df = df.iloc[1:]

    demographics_dfs.append(df)

print(demographics_dfs[0].head(5))



    

  Top Demographics              Value Percentage
0       Job titles   Technical Writer    0.25874
1       Job titles            Founder   0.024257
2       Job titles  Software Engineer   0.023306
3       Job titles    Product Manager   0.014031
4       Job titles         Co-Founder   0.013317


c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
# Merge all demographics DataFrames first
merged_demographics_df = pd.concat(demographics_dfs, ignore_index=True)

pct_col = "Percentage"
s = merged_demographics_df[pct_col].astype(str).str.strip()

# extract numeric part from:
# "< 1%", ">95%", "23%", "23.5", "< 0.5 %", etc.
s = s.str.extract(r'([0-9]+(?:\.[0-9]+)?)', expand=False)

# convert to decimal percentage
merged_demographics_df[pct_col] = pd.to_numeric(s, errors="raise") / 100

merged_demographics_df = merged_demographics_df.drop_duplicates(subset="Value")

print(merged_demographics_df.head())
print(merged_demographics_df.dtypes)

# merged_demographics_df.to_excel('demographics.xlsx', index=False)


  Top Demographics              Value  Percentage
0       Job titles   Technical Writer    0.002587
1       Job titles            Founder    0.000243
2       Job titles  Software Engineer    0.000233
3       Job titles    Product Manager    0.000140
4       Job titles         Co-Founder    0.000133
Top Demographics     object
Value                object
Percentage          float64
dtype: object


In [11]:
merged_demographics_df = pd.concat(demographics_dfs, ignore_index=True)

pct_col = "Percentage"
s = merged_demographics_df[pct_col].astype(str).str.strip()

# extract numeric part from:
# "< 1%", ">95%", "23%", "23.5", "< 0.5 %", etc.
s = s.str.extract(r'([0-9]+(?:\.[0-9]+)?)', expand=False)

# convert to decimal percentage
merged_demographics_df[pct_col] = pd.to_numeric(s, errors="raise") / 100

merged_demographics_df = merged_demographics_df.drop_duplicates(subset="Value")

print(merged_demographics_df.head())
print(merged_demographics_df.dtypes)

merged_demographics_df.to_excel('demographics.xlsx', index=False)


  Top Demographics              Value  Percentage
0       Job titles   Technical Writer    0.002587
1       Job titles            Founder    0.000243
2       Job titles  Software Engineer    0.000233
3       Job titles    Product Manager    0.000140
4       Job titles         Co-Founder    0.000133
Top Demographics     object
Value                object
Percentage          float64
dtype: object


In [12]:
len(merged_demographics_df)

31

In [13]:
df = []
for f in file_paths: 
    tmp = pd.read_excel(f, sheet_name="TOP POSTS")


    # drop first two rows, then take columns I, J, K
    subset = tmp.iloc[2:, [4,5,6]].copy()
    subset.columns = ["post_url", "post_publish_date", "impressions"]
    subset["export_file"] = f
    df.append(subset)
    print(subset.head(5))

c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


                                            post_url post_publish_date  \
2  https://www.linkedin.com/feed/update/urn:li:ac...         6/16/2025   
3  https://www.linkedin.com/feed/update/urn:li:ac...         7/19/2025   
4  https://www.linkedin.com/feed/update/urn:li:ac...         6/27/2025   
5  https://www.linkedin.com/feed/update/urn:li:ac...         6/12/2025   
6  https://www.linkedin.com/feed/update/urn:li:ac...         5/15/2025   

  impressions                                        export_file  
2       16281  data\Content_2024-11-11_2025-07-29_SteevKunduk...  
3       15160  data\Content_2024-11-11_2025-07-29_SteevKunduk...  
4       11901  data\Content_2024-11-11_2025-07-29_SteevKunduk...  
5       11696  data\Content_2024-11-11_2025-07-29_SteevKunduk...  
6       10390  data\Content_2024-11-11_2025-07-29_SteevKunduk...  
                                            post_url post_publish_date  \
2  https://www.linkedin.com/feed/update/urn:li:ac...         7/19/2025   
3  ht

In [14]:
all_posts = pd.concat(df, ignore_index=True)
# Extract the second date (export end) from export_file
# e.g. Content_2025-11-30_2025-12-15_SteevKundukulangara.xlsx -> 2025-12-15
all_posts["export_end"] = pd.to_datetime(
    all_posts["export_file"].str.extract(r"_(\d{4}-\d{2}-\d{2})_(\d{4}-\d{2}-\d{2})")[1]
)


In [15]:
# Ensure impressions is numeric
all_posts["impressions"] = pd.to_numeric(all_posts["impressions"], errors="coerce")

# Sort by export_end so latest export comes last
all_posts_sorted = all_posts.sort_values("export_end")

# Take the last row (latest export) per Post URL
latest_posts = (
    all_posts_sorted
    .groupby("post_url", as_index=False)
    .tail(1)
)

# Optional: clean final columns
posts_df = latest_posts[["post_url", "post_publish_date", "impressions", "export_end"]].copy()
posts_df = posts_df.rename(columns={"impressions": "impressions_cumulative"})
print(posts_df.head(5))
posts_df.to_excel("test.xlsx", index=False)

                                             post_url post_publish_date  \
30  https://www.linkedin.com/feed/update/urn:li:ac...          4/9/2025   
32  https://www.linkedin.com/feed/update/urn:li:ac...         2/14/2025   
33  https://www.linkedin.com/feed/update/urn:li:ac...          7/5/2025   
35  https://www.linkedin.com/feed/update/urn:li:ac...         2/22/2025   
36  https://www.linkedin.com/feed/update/urn:li:ac...         4/18/2025   

    impressions_cumulative export_end  
30                    1602 2025-07-29  
32                    1401 2025-07-29  
33                    1334 2025-07-29  
35                    1302 2025-07-29  
36                    1242 2025-07-29  


In [16]:
first_row = tmp

In [17]:
df = followers_merged.copy()

df["Date"] = pd.to_datetime(df["Date"])
df["New followers"] = pd.to_numeric(df["New followers"], errors="coerce")

df = df.sort_values("Date")

df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.to_period("M").astype(str)
df["Quarter"] = df["Date"].dt.to_period("Q").astype(str)

df["Total followers"] = df["New followers"].cumsum()


In [18]:
eng = merged_engagement_df.copy()

eng["Date"] = pd.to_datetime(eng["Date"])
eng["Impressions"] = pd.to_numeric(eng["Impressions"], errors="coerce")
eng["Engagements"] = pd.to_numeric(eng["Engagements"], errors="coerce")

eng = eng.sort_values("Date")

eng["Year"] = eng["Date"].dt.year
eng["Month"] = eng["Date"].dt.to_period("M").astype(str)
eng["Quarter"] = eng["Date"].dt.to_period("Q").astype(str)

# optional: cumulative engagements / impressions
eng["Cum engagements"] = eng["Engagements"].cumsum()
eng["Cum impressions"] = eng["Impressions"].cumsum()


In [19]:

# ========= DATA PREP =========

# Followers
df = followers_merged.copy()

df["Date"] = pd.to_datetime(df["Date"])
df["New followers"] = pd.to_numeric(df["New followers"], errors="coerce")
df = df.sort_values("Date")

df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.to_period("M").astype(str)
df["Quarter"] = df["Date"].dt.to_period("Q").astype(str)

df["Total followers"] = df["New followers"].cumsum()


In [20]:
# Engagement (impressions + engagements)
eng = merged_engagement_df.copy()

eng["Date"] = pd.to_datetime(eng["Date"])
eng["Impressions"] = pd.to_numeric(eng["Impressions"], errors="coerce")
eng["Engagements"] = pd.to_numeric(eng["Engagements"], errors="coerce")
eng = eng.sort_values("Date")

eng["Year"] = eng["Date"].dt.year
eng["Month"] = eng["Date"].dt.to_period("M").astype(str)
eng["Quarter"] = eng["Date"].dt.to_period("Q").astype(str)

eng["Cum engagements"] = eng["Engagements"].cumsum()
eng["Cum impressions"] = eng["Impressions"].cumsum()

years = sorted(df["Year"].unique())

In [21]:




# ========= APP =========

app = dash.Dash(
    __name__,
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    suppress_callback_exceptions=True,
)

app.layout = dbc.Container(
    [
        # HEADER
        dbc.Row(
            dbc.Col(
                html.Div(
                    [
                        html.H2("Audience Growth & Engagement", className="mb-0"),
                        html.P("Followers, impressions, and engagements over time", className="mb-0"),
                    ],
                    className="app-header",
                ),
                width=12,
            ),
            className="mb-4",
        ),

        # FILTERS
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label("Year", className="filter-label"),
                        dcc.Dropdown(
                            id="year-dropdown",
                            options=[{"label": int(y), "value": int(y)} for y in years],
                            value=int(years[-1]),
                            clearable=False,
                            className="filter-dropdown",
                        ),
                    ],
                    md=3,
                ),
                dbc.Col(
                    [
                        html.Label("Granularity", className="filter-label"),
                        dcc.Dropdown(
                            id="freq-dropdown",
                            options=[
                                {"label": "Daily", "value": "D"},
                                {"label": "Monthly", "value": "M"},
                                {"label": "Quarterly", "value": "Q"},
                            ],
                            value="M",
                            clearable=False,
                            className="filter-dropdown",
                        ),
                    ],
                    md=3,
                ),
            ],
            className="mb-3",
        ),

        # FOLLOWERS KPI CARDS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Total followers", className="card-title-text"),
                                html.H3(id="total-followers-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-primary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Followers this period", className="card-title-text"),
                                html.H3(id="period-followers-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-secondary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("% change vs previous", className="card-title-text"),
                                html.H3(id="pct-change-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-accent",
                    ),
                    md=4,
                ),
            ],
            className="mb-4",
        ),

        # FOLLOWERS CHARTS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Cumulative followers", className="chart-title"),
                                dcc.Graph(
                                    id="cum-line",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("New followers by period", className="chart-title"),
                                dcc.Graph(
                                    id="bar-new-followers",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
            ],
            className="mb-4",
        ),

        # ENGAGEMENT KPI CARDS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Total engagements", className="card-title-text"),
                                html.H3(id="total-engagements-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-primary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Engagements this period", className="card-title-text"),
                                html.H3(id="period-engagements-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-secondary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Engagement rate", className="card-title-text"),
                                html.H3(id="eng-rate-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-accent",
                    ),
                    md=4,
                ),
            ],
            className="mb-4",
        ),

        # IMPRESSIONS KPI CARDS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Total impressions", className="card-title-text"),
                                html.H3(id="total-impressions-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-primary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Impressions this period", className="card-title-text"),
                                html.H3(id="period-impressions-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-secondary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Impressions rate", className="card-title-text"),
                                html.H3(id="imp-rate-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-accent",
                    ),
                    md=4,
                ),
            ],
            className="mb-4",
        ),

        # ENGAGEMENT CHARTS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Engagements over time", className="chart-title"),
                                dcc.Graph(
                                    id="eng-line",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Weekdays Impressions", className="chart-title"),
                                dcc.Graph(
                                    id="eng-bar",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
            ],
            className="mb-4",
        ),

        # IMPRESSIONS CHARTS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Cumulative impressions", className="chart-title"),
                                dcc.Graph(
                                    id="imp-cum-line",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Impressions by period", className="chart-title"),
                                dcc.Graph(
                                    id="imp-bar",
                                    config={"displayModeBar": False},
                                    style={"height": "28vh"},
                                ),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
            ]
        ),
    ],
    fluid=True,
    className="app-container",
)

# ========= CALLBACK =========

@app.callback(
    [
        Output("total-followers-card", "children"),
        Output("period-followers-card", "children"),
        Output("pct-change-card", "children"),
        Output("cum-line", "figure"),
        Output("bar-new-followers", "figure"),
        Output("total-engagements-card", "children"),
        Output("period-engagements-card", "children"),
        Output("eng-rate-card", "children"),
        Output("eng-line", "figure"),
        Output("eng-bar", "figure"),
        Output("imp-cum-line", "figure"),
        Output("imp-bar", "figure"),
        Output("total-impressions-card", "children"),
        Output("period-impressions-card", "children"),
        Output("imp-rate-card", "children"),
    ],
    [Input("year-dropdown", "value"), Input("freq-dropdown", "value")],
)
def update_dashboard(year, freq):
    dff = df[df["Year"] == year].copy().sort_values("Date")
    dfe = eng[eng["Year"] == year].copy().sort_values("Date")

    # Day-of-week cumulative impressions
    dfe["Weekday"] = dfe["Date"].dt.day_name()
    dow_impressions = (
        dfe.groupby("Weekday")["Impressions"]
          .sum()
          .reindex(
              ["Monday", "Tuesday", "Wednesday", "Thursday",
               "Friday", "Saturday", "Sunday"]
          )
          .reset_index()
    )

    empty_fig = px.scatter()
    # ... rest of your code


    if dff.empty or dfe.empty:
        dash_text = "–"
        return (
            dash_text, dash_text, dash_text, empty_fig, empty_fig,
            dash_text, dash_text, dash_text, empty_fig, empty_fig,
            empty_fig, empty_fig,
            dash_text, dash_text, dash_text,
        )

    # ===== FOLLOWERS =====
    total_followers = dff["Total followers"].iloc[-1]

    if freq == "D":
        f_agg = dff.copy()
        f_x = "Date"
        f_agg = f_agg.rename(columns={"New followers": "new_followers"})
        f_new = f_agg["new_followers"].sum()
        if len(dff) > 1 and dff["Total followers"].iloc[0] != 0:
            f_pct = dff["Total followers"].iloc[-1] / dff["Total followers"].iloc[0] - 1
        else:
            f_pct = 0.0
    elif freq == "M":
        f_agg = (
            dff.groupby("Month")
               .agg(
                   new_followers=("New followers", "sum"),
                   total_followers_end=("Total followers", "last"),
               )
               .reset_index()
        )
        f_x = "Month"
        f_new = f_agg["new_followers"].iloc[-1] if not f_agg.empty else 0
        f_pct = f_agg["total_followers_end"].pct_change().iloc[-1] if len(f_agg) > 1 else 0.0
    else:
        f_agg = (
            dff.groupby("Quarter")
               .agg(
                   new_followers=("New followers", "sum"),
                   total_followers_end=("Total followers", "last"),
               )
               .reset_index()
        )
        f_x = "Quarter"
        f_new = f_agg["new_followers"].iloc[-1] if not f_agg.empty else 0
        f_pct = f_agg["total_followers_end"].pct_change().iloc[-1] if len(f_agg) > 1 else 0.0

    cum_fig = px.line(dff, x="Date", y="Total followers", markers=True)
    cum_fig.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))

    foll_bar = px.bar(f_agg, x=f_x, y="new_followers")
    foll_bar.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))
    foll_bar.update_xaxes(tickangle=-35)

    # ===== ENGAGEMENT (impressions + engagements) =====
    total_eng = dfe["Engagements"].sum()

    if freq == "D":
        e_agg = dfe.copy()
        e_x = "Date"
    elif freq == "M":
        e_agg = (
            dfe.groupby("Month")
               .agg(
                   Impressions=("Impressions", "sum"),
                   Engagements=("Engagements", "sum"),
               )
               .reset_index()
        )
        e_x = "Month"
    else:
        e_agg = (
            dfe.groupby("Quarter")
               .agg(
                   Impressions=("Impressions", "sum"),
                   Engagements=("Engagements", "sum"),
               )
               .reset_index()
        )
        e_x = "Quarter"

    period_eng = e_agg["Engagements"].iloc[-1] if not e_agg.empty else 0
    total_impr = e_agg["Impressions"].sum() if "Impressions" in e_agg.columns else dfe["Impressions"].sum()
    eng_rate = (total_eng / total_impr) if total_impr else 0.0

    eng_line = px.line(dfe, x="Date", y="Engagements", markers=True)
    eng_line.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))

    # Cumulative impressions by weekday (Mon–Sun)
    dfe["Weekday"] = dfe["Date"].dt.day_name()
    dow_impressions = (
        dfe.groupby("Weekday")["Impressions"]
        .sum()
        .reindex(["Monday", "Tuesday", "Wednesday", "Thursday",
                "Friday", "Saturday", "Sunday"])
        .reset_index()
    )

    dow_bar_fig = px.bar(dow_impressions, x="Weekday", y="Impressions")
    dow_bar_fig.update_layout(template="plotly_dark",
                            margin=dict(l=10, r=10, t=10, b=10))
    dow_bar_fig.update_xaxes(tickangle=-35)



    # ===== IMPRESSIONS =====
    imp_cum_fig = px.line(dfe, x="Date", y="Cum impressions", markers=True)
    imp_cum_fig.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))

    if freq == "D":
        imp_agg = dfe.copy()
        imp_x = "Date"
    elif freq == "M":
        imp_agg = (
            dfe.groupby("Month")
               .agg(Impressions=("Impressions", "sum"))
               .reset_index()
        )
        imp_x = "Month"
    else:
        imp_agg = (
            dfe.groupby("Quarter")
               .agg(Impressions=("Impressions", "sum"))
               .reset_index()
        )
        imp_x = "Quarter"

    imp_bar_fig = px.bar(imp_agg, x=imp_x, y="Impressions")
    imp_bar_fig.update_layout(template="plotly_dark", margin=dict(l=10, r=10, t=10, b=10))
    imp_bar_fig.update_xaxes(tickangle=-35)

    #    # Impressions KPIs
    total_impressions = total_impr
    period_impressions = (
        e_agg["Impressions"].iloc[-1] if "Impressions" in e_agg.columns and not e_agg.empty else 0
    )
    # impressions per follower (no extra *100)
    imp_rate = (total_impressions / total_followers) if total_followers else 0.0
    imp_rate_display = f"{imp_rate:0.1f}x"


    return (
    f"{int(total_followers):,}",
    f"{int(f_new):,}",
    f"{f_pct*100:0.1f}%",
    cum_fig,
    foll_bar,
    f"{int(total_eng):,}",
    f"{int(period_eng):,}",
    f"{eng_rate*100:0.2f}%",
    eng_line,
    dow_bar_fig,          # <- here, instead of weekly_bar_fig
    imp_cum_fig,
    imp_bar_fig,
    f"{int(total_impressions):,}",
    f"{int(period_impressions):,}",
    imp_rate_display,
)



if __name__ == "__main__":
    app.run(debug=True, port=8052)


In [ ]:
print(df.columns)
print(df.head())
print(df["Year"].unique())
